In [1]:
import requests
import pandas as pd
import io
from pathlib import Path

In [ ]:
# Help with querying API: https://dev.socrata.com/docs/queries/

fileName = Path('dohmh_export.csv')
if (Path.exists(fileName)):
    df = pd.read_csv(fileName)
else:
    url = 'https://data.cityofnewyork.us/resource/43nn-pn8j.csv'
    params = {
        '$limit': 500000
    }
    response = requests.get(url, params)
    csv = response.content.decode('utf-8')
    data = io.StringIO(csv)
    df = pd.read_csv(data)

df.head()

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,...,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,location_point1
0,50163690,PARIS BAGUETTE,Manhattan,990,AVENUE OF THE AMERICAS,10018.0,5169571119,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.751404,-73.986599,105.0,4.0,8400.0,1015919.0,1.008380e+09,MN17,NaN
1,50165178,Richmond Pizza Hut,Queens,11222,101ST AVE,11419.0,3475536506,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.688351,-73.831586,409.0,29.0,10800.0,4539299.0,4.094300e+09,QN54,NaN
2,41236580,DUNKIN,Queens,10005,QUEENS BOULEVARD,11375.0,7187933690,Donuts,2024-09-09T00:00:00.000,Establishment re-opened by DOHMH.,...,Cycle Inspection / Reopening Inspection,40.726899,-73.853544,406.0,29.0,71303.0,4050969.0,4.021190e+09,QN17,NaN
3,50163819,SEOUL SWEETS INC,Manhattan,308,5 AVENUE,10001.0,6467070822,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.746817,-73.985752,105.0,4.0,7600.0,1015818.0,1.008330e+09,MN17,NaN
4,50123040,MIGARBA WINE BAR,Manhattan,310,BLEECKER STREET,10014.0,9175888361,NaN,1900-01-01T00:00:00.000,NaN,...,NaN,40.732740,-74.003886,102.0,3.0,7300.0,1010017.0,1.005880e+09,MN23,NaN


In [42]:
df['cuisine_description'].unique()

array(['Donuts', 'Coffee/Tea', 'African', 'Barbecue', 'Pancakes/Waffles',
       'Sandwiches', 'Chinese', 'Southeast Asian', 'French',
       'Latin American', 'Tex-Mex', 'Mexican', 'American', 'Korean',
       'Japanese', 'Hamburgers', 'Spanish', 'Greek',
       'Bakery Products/Desserts', 'Caribbean', 'Indian',
       'Eastern European', 'Pizza', 'Italian', 'Other', 'Mediterranean',
       'Thai', 'Bangladeshi', 'Portuguese', 'Middle Eastern', 'Salads',
       'Asian/Asian Fusion', 'Chicken', 'Turkish', 'Bagels/Pretzels',
       'Brazilian', 'German', 'Peruvian',
       'Sandwiches/Salads/Mixed Buffet', 'Vegetarian', 'Seafood', 'Irish',
       'Russian', 'Jewish/Kosher', 'Frozen Desserts', 'Bottled Beverages',
       'Creole', 'Juice, Smoothies, Fruit Salads', 'Tapas', 'Filipino',
       'Pakistani', 'Chilean', 'Steakhouse', 'Vegan', 'Hotdogs/Pretzels',
       'Egyptian', 'Ethiopian', 'Moroccan', 'Australian', 'Scandinavian',
       'New American', 'Chinese/Japanese', 'Continental', 

In [39]:
boolCheck = (df['cuisine_description'].notna() & df['latitude'].notna() & df['longitude'].notna())
df['inspection_date'] = pd.to_datetime(df['inspection_date'])
df = df.drop(axis = 1,
    labels = [
        'building', 'street', 'zipcode', 'phone',
        'action', 'violation_code', 'violation_description',
        'critical_flag', 'score', 'grade', 'grade_date',
        'inspection_type', 'community_board', 'council_district',
        'census_tract', 'bin', 'bbl', 'nta', 'location_point1'
    ]
)
df = df.loc[boolCheck].copy()

In [40]:
cols = df.columns 
uniqueLocs = pd.DataFrame(
    df.groupby('camis')['inspection_date'].max()
    ).reset_index(drop = False)

unique_df = uniqueLocs.merge(df, how = 'left')

In [41]:
unique_df

,camis,inspection_date,dba,boro,cuisine_description,record_date,latitude,longitude
0,30075445,2024-11-08,MORRIS PARK BAKE SHOP,Bronx,Bakery Products/Desserts,2025-03-17T06:00:15.000,40.848231,-73.855972
1,30075445,2024-11-08,MORRIS PARK BAKE SHOP,Bronx,Bakery Products/Desserts,2025-03-17T06:00:15.000,40.848231,-73.855972
2,30075445,2024-11-08,MORRIS PARK BAKE SHOP,Bronx,Bakery Products/Desserts,2025-03-17T06:00:15.000,40.848231,-73.855972
3,30191841,2025-02-20,D.J. REYNOLDS,Manhattan,Irish,2025-03-17T06:00:15.000,40.767326,-73.984310
4,30191841,2025-02-20,D.J. REYNOLDS,Manhattan,Irish,2025-03-17T06:00:15.000,40.767326,-73.984310
...,...,...,...,...,...,...,...,...
77077,50166052,2025-03-06,EMPANADAS MONUMENTAL,Manhattan,Latin American,2025-03-17T06:00:15.000,40.854070,-73.930266
77078,50166052,2025-03-06,EMPANADAS MONUMENTAL,Manhattan,Latin American,2025-03-17T06:00:15.000,40.854070,-73.930266
77079,50166052,2025-03-06,EMPANADAS MONUMENTAL,Manhattan,Latin American,2025-03-17T06:00:15.000,40.854070,-73.930266
77080,50166052,2025-03-06,EMPANADAS MONUMENTAL,Manhattan,Latin American,2025-03-17T06:00:15.000,40.854070,-73.930266


In [43]:
unique_df.to_csv(fileName, header = True, index = False)